In [1]:
import numpy as np
import pandas as pd
import shutil
import cv2
import os


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp '/content/drive/My Drive/training_data.zip' .
!unzip training_data.zip

Streaming output truncated to the last 5000 lines.
  inflating: training_data/train/melanocytic_nevi/ISIC_0030421.jpg  
  inflating: training_data/train/melanocytic_nevi/ISIC_0030422.jpg  
  inflating: training_data/train/melanocytic_nevi/ISIC_0030425.jpg  
 extracting: training_data/train/melanocytic_nevi/ISIC_0030426.jpg  
 extracting: training_data/train/melanocytic_nevi/ISIC_0030428.jpg  
 extracting: training_data/train/melanocytic_nevi/ISIC_0030429.jpg  
 extracting: training_data/train/melanocytic_nevi/ISIC_0030430.jpg  
  inflating: training_data/train/melanocytic_nevi/ISIC_0030431.jpg  
  inflating: training_data/train/melanocytic_nevi/ISIC_0030432.jpg  
  inflating: training_data/train/melanocytic_nevi/ISIC_0030434.jpg  
 extracting: training_data/train/melanocytic_nevi/ISIC_0030435.jpg  
 extracting: training_data/train/melanocytic_nevi/ISIC_0030437.jpg  
 extracting: training_data/train/melanocytic_nevi/ISIC_0030438.jpg  
 extracting: training_data/train/melanocytic_nevi/IS

In [5]:


def train_model(model, model_save , criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            model_save.load_state_dict(best_model_wts)
            torch.save(model_save.state_dict(), '/content/drive/My Drive/training_checkpoints/train_wts_intermediate' + str(epoch) + '.pt')



        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [6]:
data_dir = "training_data"
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [7]:

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print(class_names)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))
out = torchvision.utils.make_grid(inputs)
print("Total classes : ",classes)



['basal_cell_carcinoma', 'benign_keratosis_like', 'bowen_disease', 'dermatofibroma', 'melanocytic_nevi', 'melanoma', 'vascular']
cuda:0
Total classes :  tensor([4, 4, 4, 1, 3, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4])


In [8]:
########################### MODEL ##########################################
model_ft = models.resnet101(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 7) ## fro 7 classes

model_ft = model_ft.to(device)


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [9]:
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
model_ft_save = copy.deepcopy(model_ft)
model_ft = train_model(model_ft, model_ft_save , criterion, optimizer_ft, exp_lr_scheduler,num_epochs=25)
torch.save(model_ft.state_dict(), '/content/drive/My Drive/training_checkpoints/train_wts_best.pt')



Epoch 0/24
----------
train Loss: 0.7620 Acc: 0.7328
val Loss: 0.5722 Acc: 0.7880

Epoch 1/24
----------
train Loss: 0.5804 Acc: 0.7898
val Loss: 0.5974 Acc: 0.7995

Epoch 2/24
----------
train Loss: 0.5175 Acc: 0.8200
val Loss: 0.4766 Acc: 0.8320

Epoch 3/24
----------
train Loss: 0.4525 Acc: 0.8362
val Loss: 0.4494 Acc: 0.8445

Epoch 4/24
----------
train Loss: 0.4322 Acc: 0.8471
val Loss: 0.4229 Acc: 0.8440

Epoch 5/24
----------
train Loss: 0.4026 Acc: 0.8489
val Loss: 0.4919 Acc: 0.8390

Epoch 6/24
----------
train Loss: 0.3689 Acc: 0.8619
val Loss: 0.4208 Acc: 0.8545

Epoch 7/24
----------
train Loss: 0.2923 Acc: 0.8950
val Loss: 0.3806 Acc: 0.8685

Epoch 8/24
----------
train Loss: 0.2610 Acc: 0.9085
val Loss: 0.3591 Acc: 0.8720

Epoch 9/24
----------
train Loss: 0.2491 Acc: 0.9127
val Loss: 0.3581 Acc: 0.8805

Epoch 10/24
----------
train Loss: 0.2459 Acc: 0.9124
val Loss: 0.3492 Acc: 0.8785

Epoch 11/24
----------
train Loss: 0.2347 Acc: 0.9185
val Loss: 0.3564 Acc: 0.8770

Ep

KeyboardInterrupt: ignored

In [ ]:
## Interrupted execution as I dont have a Colab Pro nd its taking sooo long per epoch